In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch.utils.data import random_split, DataLoader, TensorDataset
%matplotlib inline

import torch.optim as optim
import time
import random

# import torchvision.models as models

In [3]:
random.seed(100)

train_ds= torchvision.datasets.CIFAR10(root="data/", train = True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.CIFAR10(root="data/", train = False, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch_size=128
train_loader= DataLoader(train_ds, batch_size, shuffle =True)
test_leader=  DataLoader(test_ds, batch_size, shuffle = False)
# len(train_loader)

In [5]:
device=torch.device("cuda:1" if torch.cuda.is_available else "cpu")

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


num_classes=10
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
class Bottleneck(nn.Module):
    def __init__(self, inplane, planes, stride=1):
        super(Bottleneck,self).__init__()
        outplane=4*planes
        self.conv1 =   nn.Conv2d(inplane, planes,kernel_size=1,bias=False)  #same convolution
        self.batchN1 = nn.BatchNorm2d(planes)
        self.conv2 =   nn.Conv2d(planes, planes,kernel_size=3 ,stride=stride, padding=1 ,bias=False) #if stride is 1 then same convolution
        self.batchN2 =  nn.BatchNorm2d(planes)
        self.conv3 =   nn.Conv2d(planes, outplane,kernel_size=1,bias=False) #same convolution
        self.batchN3 =  nn.BatchNorm2d(outplane)
        
        self.shortcuts=nn.Sequential()  #creating a shortcut which will help in residual part
        if(outplane!=inplane or stride!=1):  #we make the shortcut so that the dimension match 
            self.shortcuts = nn.Sequential(nn.Conv2d(inplane, outplane, kernel_size=1,stride=stride,bias=False) ,nn.BatchNorm2d(outplane))
            
    def forward(self, x):
        out=F.relu(self.batchN1(self.conv1(x)))
        out=F.relu(self.batchN2(self.conv2(out)))
        out=self.batchN3(self.conv3(out))
        out=F.relu(out+self.shortcuts(x))   #most important step of residual network
        return out


In [7]:
class Resmodel(nn.Module):
    # def __init__(self, block=Bottleneck, no_bl=[3,4,6,3], output_no=10):
    def __init__(self, block, no_bl=[3,4,6,3], output_no=10):
        super(Resmodel,self).__init__()
        self.in_plane=64
        self.conv1= nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1 ,bias=False)
        self.batchN11= nn.BatchNorm2d(64)
        self.layer1= self._make_layer(block, 64, 3, 1)
        self.layer2= self._make_layer(block, 128, 4, 2)
        self.layer3= self._make_layer(block, 256, 6, 2)
        self.layer4= self._make_layer(block, 512, 3, 2)
        self.linear = nn.Linear(2048,10)    #finally we will classify in 10 class,hence we have 10 dimension
        
    def _make_layer(self, block, plane, numofblock, stride):
        layers=[]
        strides= [stride] + [1]*(numofblock-1) #first block will have the given stride rest will have stride=1
        #we have stride only for the first bottleneck(that too only one layer of bottleneck will use it)(hence the reduction of dimension due to stride will take place only once for one block of bottleneck)
        for st in strides:
            layers.append(Bottleneck(self.in_plane, plane, st))
            self.in_plane = plane*4     #we know that the last layer in bottleneck is 4 times the size of input layer, hence we multiply it here, so that the first layer of next bottleneck will match the dimension of last layer of previous bottleneck.
        return nn.Sequential(*layers)   #*layers will return values stored in the list
    
    def forward(self, x):
        #size of x=[batchsize,3,32,32]
        out=self.conv1(x)           #after this operation size=[batchsize,64,32,32]
        out=self.batchN11(out)
        out=F.relu(out)
        # out= F.relu(self.batchN11(self.conv1(x)))
        out=self.layer1(out)        #after this operation the output size must be [batchsize,256,32,32]
        out=self.layer2(out)        #after this operation the output size must be [batchsize,512,16,16]
        out=self.layer3(out)        #after this operation the output size must be [batchsize,1024,8,8]
        out=self.layer4(out)        #after this operation the output size must be [batchsize,2048,4,4]
        out=F.avg_pool2d(out,4)     #after this operation the output size must be [batchsize,2048,1,1]
        out= out.view(out.size(0),-1)    #after this operation the output size must be [batchsize,1,2048]
        out=self.linear(out)         #after this operation the output size must be [batchsize,1,10]
        return out


In [15]:
#try focal losss

class focalloss(nn.Module):
    def __init__(self):
        super(focalloss,self).__init__()

    def forward(self, inp,targ, alpha=1, gamma=2):
        
        # inp=F.sigmoid(inp)
        # lloss=F.cross_entropy(inp,targ,reduction='sum')
        lloss=F.cross_entropy(inp,targ)
        # inp=inp.view(-1)
        
        # targ=targ.view(-1)
        # Biloss=F.binary_cross_entropy(inp,targ, reduction='mean')
        ll_exp=torch.exp(-lloss)
        focalLoss=alpha*(1-ll_exp)**gamma * lloss
        return focalLoss


In [16]:
model = Resmodel(Bottleneck)
model=model.to(device)
# criterion = nn.CrossEntropyLoss()
criterion=focalloss()
optimizer= optim.SGD(model.parameters(), lr= 0.01)


In [17]:
%%time
n_total_steps = len(train_loader)
NOepo=2
for epoch in range(NOepo):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [128, 3, 32, 32]
        images = images.to(device)
        # labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        #for focal loss:
        # labels= F.one_hot(labels, num_classes=10)       
        # labels= labels.type(torch.FloatTensor)
        labels = labels.to(device)
        

        loss = criterion(outputs, labels)
        # break

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 16 == 0:
            print (f'Epoch [{epoch+1}/{NOepo}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# print('max value of i=',max_val_i)
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)




Epoch [1/2], Step [16/391], Loss: 2.1032
Epoch [1/2], Step [32/391], Loss: 2.3864
Epoch [1/2], Step [48/391], Loss: 1.7667
Epoch [1/2], Step [64/391], Loss: 1.9700
Epoch [1/2], Step [80/391], Loss: 1.5461
Epoch [1/2], Step [96/391], Loss: 1.5713
Epoch [1/2], Step [112/391], Loss: 1.6566
Epoch [1/2], Step [128/391], Loss: 1.4139
Epoch [1/2], Step [144/391], Loss: 1.7263
Epoch [1/2], Step [160/391], Loss: 1.2902
Epoch [1/2], Step [176/391], Loss: 1.3191
Epoch [1/2], Step [192/391], Loss: 1.3928
Epoch [1/2], Step [208/391], Loss: 1.0788
Epoch [1/2], Step [224/391], Loss: 1.2690
Epoch [1/2], Step [240/391], Loss: 0.9117
Epoch [1/2], Step [256/391], Loss: 1.0811
Epoch [1/2], Step [272/391], Loss: 1.0344
Epoch [1/2], Step [288/391], Loss: 0.8762
Epoch [1/2], Step [304/391], Loss: 1.0268
Epoch [1/2], Step [320/391], Loss: 0.9609
Epoch [1/2], Step [336/391], Loss: 1.1074
Epoch [1/2], Step [352/391], Loss: 1.0937
Epoch [1/2], Step [368/391], Loss: 0.7280
Epoch [1/2], Step [384/391], Loss: 0.904

In [18]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_leader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # for i in range(batch_size):
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 53.09 %
Accuracy of plane: 66.0 %
Accuracy of car: 88.0 %
Accuracy of bird: 26.1 %
Accuracy of cat: 64.9 %
Accuracy of deer: 35.5 %
Accuracy of dog: 25.3 %
Accuracy of frog: 64.5 %
Accuracy of horse: 66.1 %
Accuracy of ship: 63.4 %
Accuracy of truck: 31.1 %
